# Final Project 
## Raw Analysis

In [10]:
# Import all the needed modules for this exercise
import pandas as pd
import country_converter as coco
import requests # For downloading the website
from bs4 import BeautifulSoup # For parsing the website
import warnings # import warnings 
warnings.filterwarnings("ignore") # ignore global warnings
import time # To put the system to sleep
import random # for random numbers
import sqlite3
import matplotlib.pyplot as pt
from plotnine import *
import numpy as np
import openpyxl


In [25]:
# Load first dataset (Source: https://www.kff.org/statedata/custom-state-report/?32139-year-=2019&view=3&i=32148~460185~32141~32139~32132~32136&g=us~al~ak~az~ar~ca~co~ct~de~dc~fl~ga~hi~id~il~in~ia~ks~ky~la~me~md~ma~mi~mn~ms~mo~mt~ne~nv~nh~nj~nm~ny~nc~nd~oh~ok~or~pa~ri~sc~sd~tn~tx~ut~vt~va~wa~wv~wi~wy)
Poverty_Expenditure = pd.read_excel('/Users/abhatti/Documents/masterInScience/PPOL564/Final Project/data/statehealthfacts.xlsx')
dta = Poverty_Expenditure.melt(id_vars="Unnamed: 0")
dta = dta.rename(columns={"Unnamed: 0": "Features", "variable": "Country"})
pv = dta.pivot_table(index=["Country"], 
                    columns='Features', 
                    values='value')
Poverty_Expenditure = pv

In [27]:
Poverty_Expenditure.head()

Features,Poverty Rate by Age: 65+,Poverty Rate by Age: Adults 19-64,Poverty Rate by Age: Children 0-18,Poverty Rate by Age: Total,Poverty Rate by Race/Ethnicity: American Indian/Alaska Native,Poverty Rate by Race/Ethnicity: Asian/Native Hawaiian and Pacific Islander,Poverty Rate by Race/Ethnicity: Black,Poverty Rate by Race/Ethnicity: Hispanic,Poverty Rate by Race/Ethnicity: Multiple Races,Poverty Rate by Race/Ethnicity: Total,Poverty Rate by Race/Ethnicity: White,Total State Expenditures (in millions)
Country,,,,,,,,,,,,
Alabama,0.105,0.148,0.208,0.154,0.156,0.107,0.236,0.244,0.186,0.154,0.115,28834.0
Alaska,0.083,0.101,0.133,0.107,0.240,NaN,NaN,0.149,0.069,0.107,0.069,11148.0
Arizona,0.091,0.126,0.187,0.134,0.290,0.115,0.175,0.182,0.153,0.134,0.092,38691.0
Arkansas,0.103,0.156,0.230,0.165,0.155,0.125,0.289,0.201,0.227,0.165,0.134,25639.0
California,0.104,0.106,0.157,0.118,0.203,0.090,0.193,0.150,0.107,0.118,0.084,300445.0


In [75]:
# Load unemployment data (12-Month Data (4/1/2020 – 03/31/2021)
# Improper payment data for the Unemployment Insurance (UI) program derived from the Benefit Accuracy Measurement (BAM) program for the 12-month period ending March 31, 2021. BAM is a quality control statistical survey used to identify errors and support corrective action in the state unemployment insurance (UI) system. Data includes improper payment percentages, total dollar amounts, and root cause by state.
Unemployment_Data = pd.read_excel('/Users/abhatti/Documents/masterInScience/PPOL564/Final Project/data/2021 - Twelve Months Ending March 31.xlsx', sheet_name='Improper Payment Rates', header = 6, skipfooter = 6)
Unemployment_Data = Unemployment_Data.rename(columns={"ST": "State", 
                                                      "Amount Paid": "Total Unemployment Paid", 
                                                      "(a)": "Overpayment Rate", 
                                                      "(c)": "Amount Overpaid", 
                                                      "(b)": "Underpayment Rate", 
                                                      "(d)": "Amount Underpaid", 
                                                      "(a)+(b)": "Improper Payment Rate", 
                                                      "(c)+(d)": "Amount Improperly Paid", 
                                                      "Work. Search.": "Overpayment Rate excl Work Search",
                                                      "Work. Search..1": "Amount Overpaid excl Work Search",
                                                      "Rate": "Fraud Rate"
                                                     })
Unemployment_Data.head()

,State,Total Unemployment Paid,Overpayment Rate,Amount Overpaid,Underpayment Rate,Amount Underpaid,Improper Payment Rate,Amount Improperly Paid,Overpayment Rate excl Work Search,Amount Overpaid excl Work Search,Fraud Rate
0,AK,314026879,0.08134,25542322,0.04325,13582376,0.12459,39124699,0.06550,20569016,0.01785
1,AL,533312996,0.23671,126239472,0.00285,1519078,0.23956,127758550,0.23529,125482820,0.13475
2,AR,171995172,0.22442,38598834,0.01726,2968952,0.24168,41567786,0.22442,38598834,0.10602
3,AZ,919184394,0.25343,232951241,0.00000,0,0.25343,232951241,0.25343,232951241,0.14078
4,CA,16583022143,0.07375,1222937384,0.01559,258463590,0.08933,1481400974,0.07375,1222937384,0.03080


In [86]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

# Revsie the diction
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [95]:
abbrev_to_us_state = pd.DataFrame.from_dict(abbrev_to_us_state, orient='index')

TypeError: 'numpy.ndarray' object is not callable

In [99]:
abbrev_to_us_state = abbrev_to_us_state.rename(columns={"0": "State"})

In [109]:
abbrev_to_us_state.columns[0]

0

In [106]:
abbrev_to_us_state

,0
AL,Alabama
AK,Alaska
AZ,Arizona
AR,Arkansas
CA,California
CO,Colorado
CT,Connecticut
DE,Delaware
FL,Florida
GA,Georgia


In [87]:
Unemployment_Data.merge(abbrev_to_us_state, on='user_id', how='left')

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'DC': 'District of Columbia

In [78]:
# define the url and ready data
url = "https://www.usaspending.gov/state"


In [79]:
def usaspending_scraper(url=None):

    # Download the webpage
    page = requests.get(url)

    # If a connection was reached
    if page.status_code == 200:

        # Parse
        soup = BeautifulSoup(page.content, 'html.parser')

        # Pull Table
        table = soup.select("#main-content > div > div:nth-child(2) > div.state-landing__results > table")[0].get_text()

        # Return data
        return [table]


In [81]:
page = requests.get(url)

In [84]:
soup = BeautifulSoup(page.content, 'html.parser')

In [85]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="/"/>
<!-- Facebook Open Graph -->
<meta content="website" property="og:type">
<!-- Twitter Card Tags -->
<meta content="summary_large_image" name="twitter:card">
<meta name="twitter:domain" value="usaspending.gov"/>
<meta name="twitter:site" value="@usaspending"/>
<title>USAspending.gov</title>
<link href="https://api.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.css" rel="stylesheet"/>
<link href="img/favicon.ico" rel="shortcut icon"/>
<script async="" id="_fed_an_ua_tag" src="https://dap.digitalgov.gov/Universal-Federated-Analytics-Min.js?agency=TREAS&amp;subagency=FS" type="text/javascript"></script>
<script src="https://api.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>
<!-- Google Analytics -->
<script>
        window.ga = window.ga || function () { (ga.q = ga.q || []).push(arguments) }; ga.l = +new Date;
        ga('create', 'UA-6